In [ ]:
import pandas as pd
import numpy as np

In [ ]:
results = pd.read_csv('../results/e4/ng_religion_v3.csv', index_col=0)
results.head()

In [ ]:
def convert_to_list(liststring):
    return [int(i) for i in liststring[1:-1].replace('\n', '').split()]

In [ ]:
rm = results.pivot_table(values='IoU', 
                    index='method_segment', 
                    columns='method_gt', 
                    aggfunc='mean')
rm

In [ ]:
def generate_latex_results(results, decimals=2, do_htmm=True):
    columns = ['segment_neural', 'segment_topictiling', 'segment_sentence']
    if do_htmm:
        columns.append('segment_htmm')
    rm = results.pivot_table(values='IoU', 
                            index='method_segment', 
                            columns='method_gt', 
                            aggfunc='mean')
    rs = results.pivot_table(values='IoU', 
                            index='method_segment', 
                            columns='method_gt', 
                            aggfunc='std')
    res = (rm.round(decimals).astype('str')+'('+rs.round(decimals).astype('str')+')')
    res = res.reset_index()
    order_dict = {
        'segment_neural' : 0, 
        'segment_paragraph' : 4,
        'segment_sentence' : 3,
        'segment_topictiling' : 1,
        'segment_htmm' : 2
        
    }
    res['order'] = res.apply(lambda row: order_dict[row.method_segment], axis=1)
    res = res.sort_values('order')
    res = res.set_index('method_segment')
    return res[columns]

In [ ]:
print(generate_latex_results(results, do_htmm=True).to_latex())

In [ ]:
results.ground_truth = results.apply(lambda row: convert_to_list(row.ground_truth), axis=1)


In [ ]:
results.prediciton = results.apply(lambda row: convert_to_list(row.prediciton), axis=1)

In [ ]:
import numpy as np
def detection(true, pred):
    if len(true)==len(pred):
        return int((np.array(true)*np.array(pred)).sum()>0)
    else:
        return float('NaN')

In [ ]:
results['detection'] = results.apply(lambda row: detection(row.ground_truth,row.prediciton),axis=1)

In [ ]:
r2 = results.loc[results.detection==1]

In [ ]:
r2.pivot_table(values='IoU', 
                    index='method_segment', 
                    columns='method_gt', 
                    aggfunc='mean')

In [ ]:
results['np'] = results.apply(lambda row: sum(row.prediciton), axis=1)

In [ ]:
results.loc[results.np==0]

The prediction may be 0 if a segment is detected which does not contain any sentece, notably in paragraph segmentation where more than 1 newline may be present (does not happen for other segmentations)

In [ ]:
from nltk.tokenize import sent_tokenize

In [ ]:
sent_tokenize('\n')

In [ ]:
sent_tokenize('ih')

In [ ]:
sent_tokenize(' ')